In [95]:
import pandas as pd
import numpy as np
from preprocessor import classify_features

In [96]:
# Variables
USED_COLUMNS = []
UNUSED_COLUMNS = []

## Introductions to Dataset
-----------
- Uploading_Attack : [Uploading Attack](https://velog.io/@dhk22/Kali-Linux-DVWA%EC%8B%A4%EC%8A%B5-File-Upload-Attack)
- Password Attack : [Password Attack](https://medium.com/hacker-toolbelt/dvwa-1-9-ii-brute-force-4cd8efe1c7f2)  = Brute Force Attack
- SQL Injection Attack : [SQL injection Attack](https://velog.io/@jsw4215/DVWA%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-SQL-INJECTION-%EC%8B%A4%EC%8A%B5)
- XSS Attack : [XSS Attack](https://velog.io/@dhk22/Kali-Linux-DVWA%EC%8B%A4%EC%8A%B5-XSS)

### Why some attacks has been not added?
Due to problem of difficulty at detecting no-uri included attack, we have excluded the dataset of these list.
- Backdoor Attack
- OS Fingerprinting Attack
- Port Scanning Attack


In [ ]:
# 
import os
csvList = [_ for _ in os.listdir("Full-Dataset") if _.endswith(".csv")]
datasets = [pd.read_csv(os.path.join("Full-Dataset", _)) for _ in csvList]
concat_data = pd.concat(datasets)
"""
The manual import of the files is the code below.
"""
# Password_Attack = pd.read_csv("Full-Dataset/Password_attack.csv")
# SQL_injection = pd.read_csv("Full-Dataset/SQL_injection_attack.csv")
# XSS_attack = pd.read_csv("Full-Dataset/XSS_attack.csv")
# Uploading_attack = pd.read_csv("Full-Dataset/Uploading_attack.csv")

In [98]:
# Check the columns
for name, columns in zip(csvList, [list(_.columns) for _ in datasets]):
    print(f"{name=} {columns=}")

name='Password_attack.csv' columns=['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4', 'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum', 'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused', 'http.file_data', 'http.content_length', 'http.request.uri.query', 'http.request.method', 'http.referer', 'http.request.full_uri', 'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options', 'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream', 'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in', 'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.msgtype', 'mqtt.pr

In [124]:
common_columns = []
uncommon_columns = []

all_columns = []
for _ in datasets:
    all_columns = [*all_columns, *list(_.columns)]
for col in set(all_columns):
    column_array = [list(_.columns) for _ in datasets]
    all_columns = [(col in _) for _ in column_array]
    if all(all_columns):
        common_columns.append(col)
    else: 
        uncommon_columns.append(col)
uncommon_columns # There is only shared columns to all the datas.

[]

In [125]:
# Check the columns that has the only unique value.
cols, unused = classify_features(concat_data, ["Attack_type"])
cols, unused

(['ip.src_host',
  'frame.time',
  'http.request.uri.query',
  'tcp.checksum',
  'http.response',
  'arp.src.proto_ipv4',
  'http.request.method',
  'http.content_length',
  'Attack_type',
  'http.request.full_uri',
  'arp.dst.proto_ipv4',
  'tcp.connection.rst',
  'arp.hw.size',
  'tcp.len',
  'tcp.seq',
  'tcp.options',
  'tcp.connection.syn',
  'http.referer',
  'ip.dst_host',
  'tcp.flags.ack',
  'tcp.ack_raw',
  'tcp.connection.fin',
  'tcp.dstport',
  'tcp.flags',
  'http.request.version',
  'tcp.ack',
  'http.file_data',
  'tcp.connection.synack',
  'tcp.payload',
  'tcp.srcport',
  'arp.opcode'],
 ['Attack_label',
  'mqtt.topic_len',
  'dns.retransmit_request_in',
  'mqtt.conflags',
  'mqtt.msgtype',
  'icmp.seq_le',
  'dns.retransmission',
  'mqtt.conflag.cleansess',
  'udp.port',
  'dns.retransmit_request',
  'mqtt.topic',
  'dns.qry.qu',
  'mbtcp.len',
  'mbtcp.trans_id',
  'dns.qry.name',
  'mqtt.proto_len',
  'mqtt.hdrflags',
  'dns.qry.type',
  'mbtcp.unit_id',
  'icmp.ch

In [126]:
CHECK_HOW_MANY_TYPES_OF_DATAS_ARE_INCLUDED = 2
concat = concat_data
for col in cols:
    if concat[col].unique().shape[0] > CHECK_HOW_MANY_TYPES_OF_DATAS_ARE_INCLUDED: continue
    print(col, concat[col].unique())

http.response [0. 1.]
tcp.connection.rst [0. 1.]
arp.hw.size [0. 6.]
tcp.connection.syn [1. 0.]
tcp.flags.ack [0. 1.]
tcp.connection.fin [0. 1.]
tcp.connection.synack [0. 1.]


In [104]:
import json
with open("column_data.txt", "w") as f:
    f.write(json.dumps({"Used_Columns":cols, "Unused_Columns":unused}, indent=2))

In [127]:
concat_datas = pd.DataFrame({col:concat_data[col] for col in cols})
processed_data = concat_datas

In [128]:
import json
with open("unique_values.txt", "w") as f:
    data = {}
    for col in cols:
        count = processed_data[col].unique().shape[0]
        unique_cols = list(processed_data[col].unique()) if count < 10 else list(processed_data[col].unique()[:10])
        data[col] = {
            "count": count,
            "unique_columns": unique_cols
        }
    f.write(json.dumps(data, indent=2, ensure_ascii=True))

In [129]:
temp = processed_data.loc[(processed_data["http.request.method"] != '0')]

In [130]:
# temp_cols = temp.columns
# used = list(temp_cols[:])
# unused = []
# for col in temp.columns:
#     if not all(temp[col]) or temp[col].unique().shape[0] <= 1 and col != 'Attack_type':
#         used.remove(col)
#         unused.append(col)
used, unused = classify_features(temp, ["Attack_type"])

In [131]:
USED_COLUMNS = used
UNUSED_COLUMNS = set([*UNUSED_COLUMNS, *unused])
used, unused

(['frame.time',
  'http.request.uri.query',
  'tcp.checksum',
  'http.request.method',
  'arp.src.proto_ipv4',
  'http.content_length',
  'Attack_type',
  'http.request.full_uri',
  'arp.dst.proto_ipv4',
  'tcp.len',
  'tcp.seq',
  'tcp.options',
  'http.referer',
  'tcp.ack_raw',
  'http.request.version',
  'tcp.ack',
  'http.file_data',
  'tcp.payload',
  'tcp.srcport'],
 ['ip.src_host',
  'ip.dst_host',
  'tcp.flags.ack',
  'tcp.connection.rst',
  'tcp.connection.synack',
  'arp.hw.size',
  'http.response',
  'tcp.connection.fin',
  'tcp.dstport',
  'tcp.flags',
  'arp.opcode',
  'tcp.connection.syn'])

In [132]:
import json
with open("column_data.txt", "w") as f:
    f.write(json.dumps({"USED_COLUMNS":USED_COLUMNS, "UNUSED_COLUMNS":list(UNUSED_COLUMNS)}, indent=2, ensure_ascii=True))    

In [133]:
temp = temp.drop(unused, axis="columns")

In [134]:
temp[:15]

,frame.time,http.request.uri.query,tcp.checksum,arp.src.proto_ipv4,http.request.method,http.content_length,Attack_type,http.request.full_uri,arp.dst.proto_ipv4,tcp.len,tcp.seq,tcp.options,http.referer,tcp.ack_raw,http.request.version,tcp.ack,http.file_data,tcp.payload,tcp.srcport
4,2021 01:45:35.974758000,0.0,0x0000016d,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,0101080a9a8a02a2d3a23932,0.0,2.788267e+09,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37670.0
6,2021 01:45:35.976362000,0.0,0x000067c9,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,0101080a9a8a02a3d3a23935,0.0,3.173240e+08,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37672.0
31,2021 01:45:36.094926000,0.0,0x000057a1,0,POST,37.0,Password,http://192.168.0.128/DVWA/login.php,0,260.0,1.0,0101080a9a8a031bd3a239b3,0.0,9.393408e+08,HTTP/1.0,1.0,username=admin&password=0&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37674.0
33,2021 01:45:36.095802000,0.0,0x0000964a,0,POST,38.0,Password,http://192.168.0.128/DVWA/login.php,0,261.0,1.0,0101080a9a8a031cd3a239b3,0.0,3.983958e+09,HTTP/1.0,1.0,username=admin&password=00&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37676.0
55,2021 01:45:36.209790000,0.0,0x0000ce94,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,221.0,1.0,0101080a9a8a038ed3a23a26,0.0,6.664912e+08,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37678.0
57,2021 01:45:36.213232000,0.0,0x00000e8a,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,221.0,1.0,0101080a9a8a038ed3a23a26,0.0,4.075773e+09,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37680.0
94,2021 01:45:37.143631000,0.0,0x00009e3e,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,0101080a9a8a0737d3a23dc4,0.0,3.017786e+09,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37688.0
96,2021 01:45:37.144421000,0.0,0x0000ce06,0,GET,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,0101080a9a8a0738d3a23dc6,0.0,1.187477e+09,HTTP/1.0,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37690.0
120,2021 01:45:37.258890000,0.0,0x00005130,0,POST,37.0,Password,http://192.168.0.128/DVWA/login.php,0,260.0,1.0,0101080a9a8a07abd3a23e3f,0.0,3.692066e+09,HTTP/1.0,1.0,username=admin&password=0&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37692.0
122,2021 01:45:37.260781000,0.0,0x0000872f,0,POST,38.0,Password,http://192.168.0.128/DVWA/login.php,0,261.0,1.0,0101080a9a8a07abd3a23e3f,0.0,4.179367e+09,HTTP/1.0,1.0,username=admin&password=00&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37694.0


In [135]:
Encodings = {
    key: pd.get_dummies(temp[key])
        for key in ["http.request.method", "http.request.version"]
}

In [136]:
list(Encodings.keys())

['http.request.method', 'http.request.version']

In [137]:
temp = pd.DataFrame({
    **{col: temp[col]
        for col in (_ for _ in temp.columns if _ not in Encodings.keys())},
    **{col: Encodings[key][col]
                for key in Encodings.keys()
                    for col in Encodings[key].columns 
    }, 
})

In [138]:
temp.head()

,frame.time,http.request.uri.query,tcp.checksum,arp.src.proto_ipv4,http.content_length,Attack_type,http.request.full_uri,arp.dst.proto_ipv4,tcp.len,tcp.seq,...,tcp.ack_raw,tcp.ack,http.file_data,tcp.payload,tcp.srcport,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
4,2021 01:45:35.974758000,0.0,0x0000016d,0,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,...,2.788267e+09,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37670.0,True,False,False,True,False
6,2021 01:45:35.976362000,0.0,0x000067c9,0,0.0,Password,http://192.168.0.128/DVWA/login.php,0,86.0,1.0,...,3.173240e+08,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37672.0,True,False,False,True,False
31,2021 01:45:36.094926000,0.0,0x000057a1,0,37.0,Password,http://192.168.0.128/DVWA/login.php,0,260.0,1.0,...,9.393408e+08,1.0,username=admin&password=0&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37674.0,False,True,False,True,False
33,2021 01:45:36.095802000,0.0,0x0000964a,0,38.0,Password,http://192.168.0.128/DVWA/login.php,0,261.0,1.0,...,3.983958e+09,1.0,username=admin&password=00&Login=Login,504f5354202f445657412f6c6f67696e2e706870204854...,37676.0,False,True,False,True,False
55,2021 01:45:36.209790000,0.0,0x0000ce94,0,0.0,Password,http://192.168.0.128/DVWA/login.php,0,221.0,1.0,...,6.664912e+08,1.0,0,474554202f445657412f6c6f67696e2e70687020485454...,37678.0,True,False,False,True,False


In [139]:
temp.to_csv("Full-Dataset/processed/method_na_removed.csv", sep=",", index=False)

In [140]:
temp.count()

frame.time                78697
http.request.uri.query    78697
tcp.checksum              78697
arp.src.proto_ipv4        78697
http.content_length       78697
Attack_type               78697
http.request.full_uri     78697
arp.dst.proto_ipv4        78697
tcp.len                   78697
tcp.seq                   78697
tcp.options               78697
http.referer              78697
tcp.ack_raw               78697
tcp.ack                   78697
http.file_data            78697
tcp.payload               78697
tcp.srcport               78697
GET                       78697
POST                      78697
TRACE                     78697
HTTP/1.0                  78697
HTTP/1.1                  78697
dtype: int64

In [141]:
temp.groupby(["Attack_type"])["Attack_type"].count()

Attack_type
Password         71035
SQL_injection     4379
Uploading         1907
XSS               1376
Name: Attack_type, dtype: int64

In [142]:
temp.columns

Index(['frame.time', 'http.request.uri.query', 'tcp.checksum',
       'arp.src.proto_ipv4', 'http.content_length', 'Attack_type',
       'http.request.full_uri', 'arp.dst.proto_ipv4', 'tcp.len', 'tcp.seq',
       'tcp.options', 'http.referer', 'tcp.ack_raw', 'tcp.ack',
       'http.file_data', 'tcp.payload', 'tcp.srcport', 'GET', 'POST', 'TRACE',
       'HTTP/1.0', 'HTTP/1.1'],
      dtype='object')